In [40]:
import sys
import pandas as pd
import numpy as np
sys.path.append('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
from backend.preprocess.preprocess import main as load_data
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import os
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [41]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [42]:
# Load data
df = load_data()

last_season = df[[index[0].year >= 2021 for index in df.index]]
df = df[[index[0].year < 2021 for index in df.index]]
X = df.drop(['y'], axis=1)
y = df[['y']]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

# Pipeline
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('nn', MLPClassifier(random_state=1, activation='identity'))
    ]
)


Executing load_data ...
	Done: 4.1897s
Executing preprocess ...
	Done: 2.0799s
Executing load_target_data ...


c:\Python38\lib\site-packages\pandas\core\window\rolling.py:2010: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(


	Done: 0.8660s
Executing merge_x_y ...
	Done: 0.0360s


In [43]:
parameters = {
    'nn__hidden_layer_sizes': [
        (100), (400), (500), 
        (100, 100, 100), (400, 400), (500, 500)
    ],
    'nn__activation': ['identity']
}

In [44]:
search = GridSearchCV(
    pipe,
    parameters,
    scoring='accuracy',
    n_jobs=-1,
    cv=KFold(n_splits=5, shuffle=True, random_state=1)
).fit(X_train, y_train)

In [45]:
results_df = pd.DataFrame(search.cv_results_)
results_df.sort_values(by=['rank_test_score'], inplace=True)
results_df.set_index(['param_nn__activation', 'param_nn__hidden_layer_sizes'], inplace=True)
results_df[['mean_test_score', 'mean_test_score', 'rank_test_score']]

mean_test_score  \
param_nn__activation param_nn__hidden_layer_sizes                    
identity             (500, 500)                           0.614894   
                     100                                  0.614362   
                     500                                  0.605851   
                     (100, 100, 100)                      0.605319   
                     (400, 400)                           0.604787   
                     400                                  0.600000   

                                                   mean_test_score  \
param_nn__activation param_nn__hidden_layer_sizes                    
identity             (500, 500)                           0.614894   
                     100                                  0.614362   
                     500                                  0.605851   
                     (100, 100, 100)                      0.605319   
                     (400, 400)                           0.604787   
                     400                                  0.600000   

                                                   rank_test_score  
param_nn__activation param_nn__hidden_layer_sizes                   
identity             (500, 500)                                  1  
                     100                                         2  
                     500                                         3  
                     (100, 100, 100)                             4  
                     (400, 400)                                  5  
                     400                                         6

In [46]:
search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('nn',
                 MLPClassifier(activation='identity',
                               hidden_layer_sizes=(500, 500),
                               random_state=1))])

In [47]:
scores = cross_val_score(
    search.best_estimator_, 
    X_train, y_train, 
    cv=KFold(n_splits=5, shuffle=True, random_state=1)
)
print(f'Mean: {scores.mean()} Std: {scores.std()}')
print(search.best_estimator_.score(X_test, y_test))

Mean: 0.6148936170212765 Std: 0.003908760227845493
0.6106382978723405
